# Classify motor types by sound
In production environment like a warehouse distribution center, there are hundreds of AC motors that are driving conveyor belts and sorters day and night. Let’s say one motor at a critical link breaks down, it can cause major downtime of the whole system. An experienced maintenance engineer can identify a faulty motor by listening to its sounds and take action to correct it before it is too late. 

In this demo, we are going to train our model to be an expert. It can tell if a motor is faulty by listening to its sound.

Using MFCC features as model input, the mel-frequency cepstrum (MFC) is a representation of the short-term power spectrum of a sound, Mel-frequency cepstral coefficients (MFCCs) are coefficients that collectively make up an MFC, [MFC wiki](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)

In [1]:
import glob
import pandas as pd
from scipy import stats
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline
plt.style.use('ggplot')

# About the wav files

### Signal recording location
- de: motor drive end
- re: motor rear end

### Type of motors
- new: a new motor
- used: a used motor, still functional
- red: an identified bad motor

e.g. **used_60Hz_re_10s_22khz.wav** means, 
- an used motor
- VFD frequency is 60Hz
- microphone attached to the rear end of the motor housing
- recorded for 10 seconds
- the sampling rate is 22Khz

In [2]:
base_dir = "./data/"
sound_file_paths = ["new_60Hz_de_10s_22khz.wav","new_60Hz_re_10s_22khz.wav",
                    "used_60Hz_de_10s_22khz.wav","used_60Hz_re_10s_22khz.wav",
                    "red_60Hz_de_10s_22khz.wav","red_60Hz_re_10s_22khz.wav"]
# Output tags
sound_names = ["new","new",
               "used","used",
               "red","red"]

# Function to extract MFCC feature out of WAV files for model input

- frames: 41, one frame consists of 512 samples
- hop length: 512 is, the number of samples between successive **frames**
- window_size: 512 * (41-1) = 20480. Total samples to compute the MFCCs features. Given sampling rate 22kHz, window time about 1 second.



In [3]:
def feature_normalize(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu) / sigma
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield int(start), int(start + window_size)
        start += (window_size / 2) # stepping at half window size
def extract_features(base_dir, sound_file_paths, sound_names ,bands = 20, frames = 41):
    window_size = 512 * (frames - 1)
    mfccs = []
    labels = []
    for i, sound_file_path in enumerate(sound_file_paths):
        sound_file_full_path = os.path.join(base_dir, sound_file_path)
        sound_clip,s = librosa.load(sound_file_full_path)
        sound_clip = feature_normalize(sound_clip)
        label = sound_names[i]
        for (start,end) in windows(sound_clip,window_size):
            if(len(sound_clip[start:end]) == window_size):
                signal = sound_clip[start:end]
                # y: audio time series, sr: sampling rate, n_mfcc: number of MFCCs to return
                # librosa.feature.mfcc() function return numpy array with shape (bands, frames)
                # transpose since the model expects time axis(frames) come first
                mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T 
                mfccs.append(mfcc)
                labels.append(label)
    features = np.asarray(mfccs)
    return np.array(features), np.array(labels,dtype = np.str)

def one_hot_encode(labels):
    return np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [4]:
bands = 20
frames = 41
features,labels = extract_features(base_dir, sound_file_paths, sound_names, bands = bands, frames = frames)
labels = one_hot_encode(labels)

shape of **features**
(number of samples,frames,bands)

In [5]:
features.shape

(120, 41, 20)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.2, random_state=10)

# Build model

In [7]:
learning_rate = 0.01
training_iters = 300
batch_size = 5
display_step = 100

# Network Parameters
n_input = bands 
n_steps = frames
n_hidden = 64
n_classes = 3 # new , other , red

In [8]:
tf.reset_default_graph()
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

weight = tf.Variable(tf.random_normal([n_hidden, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

## RNN
paras
- x: The RNN inputs for **tf.nn.dynamic_rnn**.
- weight: softmax fully connected layer weight
- bias: softmax fully connected layer bias

In [9]:
def lstm_cell(lstm_size, keep_prob):
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    return tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob)
def RNN(x, weight, bias, keep_prob, num_layers=2):
    lstm_layers = tf.contrib.rnn.MultiRNNCell([lstm_cell(n_hidden, keep_prob) for _ in range(num_layers)])
    output, state = tf.nn.dynamic_rnn(lstm_layers, x, dtype = tf.float32)
    output = tf.transpose(output, [1, 0, 2])
    last = tf.gather(output, int(output.get_shape()[0]) - 1)
    return tf.nn.softmax(tf.matmul(last, weight) + bias)

In [10]:
prediction = RNN(x, weight, bias, keep_prob)
# Define loss and optimizer
loss_f = -tf.reduce_sum(y * tf.log(prediction))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_f)
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

c:\users\hasee\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# Training

In [11]:
saver = tf.train.Saver()
session=tf.InteractiveSession()
# Initializing the variables
session.run(tf.global_variables_initializer())

for itr in range(training_iters):    
    offset = (itr * batch_size) % (labels.shape[0] - batch_size)
    batch_x = X_train[offset:(offset + batch_size), :, :]
    batch_y = y_train[offset:(offset + batch_size), :]
    _, c = session.run([optimizer, loss_f],feed_dict={x: batch_x, y : batch_y, keep_prob: 0.95})

    if itr % display_step == 0:
        # Calculate batch accuracy
        acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
        # Calculate batch loss
        loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
        print("Iter " + str(itr) + ", Minibatch Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc))

print('Test accuracy: ',round(session.run(accuracy, feed_dict={x: X_test, y: y_test, keep_prob: 1}) , 3))
saver.save(session, save_path = "./model/acoustic.ckpt")

Iter 0, Minibatch Loss= 2.890719, Training Accuracy= 0.80000
Iter 100, Minibatch Loss= 0.000040, Training Accuracy= 1.00000
Iter 200, Minibatch Loss= 0.000072, Training Accuracy= 1.00000
Test accuracy:  1.0


'./model/acoustic.ckpt'

## Test new wav file
**new_60Hz.wav** we test here is recorded with a new motor

In [12]:
test_features,_ = extract_features('./data', ["test_new_60Hz.wav"], ["Unknown"])

In [13]:
y_predicts = session.run(prediction, feed_dict={x: test_features, keep_prob: 1})
LABELS = ["new","used","red"]
predicted_logit = stats.mode(np.argmax(y_predicts,1))[0][0]
predicted_label = LABELS[predicted_logit]
predicted_probability = stats.mode(np.argmax(y_predicts,1))[1][0] / len(y_predicts)

In [14]:
(predicted_label, predicted_probability)

('new', 1.0)